<a href="https://colab.research.google.com/github/vishnuanil122/Bert-Binary-Text-Classifier/blob/master/vgg_augmented_batch_normalised_conv_added.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/

**Based on the above link**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/Colab\ Notebooks/VGGFACEID/

/content/gdrive/My Drive/Colab Notebooks/VGGFACEID


In [1]:
from keras import metrics
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalMaxPooling2D,Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape = (224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.





58892288/58889256 [==============================] - 2s 0us/step








In [0]:
# x=base_model.output
x = base_model.get_layer('block5_conv3').output

x=Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# x=Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
# x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x=Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
# x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

x = Flatten(input_shape=base_model.output_shape[1:])(x)
x=Dense(256,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(256,activation='relu')(x) #dense layer 2
x=Dense(256,activation='relu')(x) #dense layer 3
x=Dense(19,activation='softmax')(x) #final layer with softmax activation
model_final = Model(inputs=base_model.input, outputs=x)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
for layer in model_final.layers[:17]:
    layer.trainable=False
for layer in model_final.layers[17:]:
    layer.trainable=True

In [0]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

In [0]:
batch_size = 32 

In [0]:
epochs = 100

In [0]:
train_generator=train_datagen.flow_from_directory('data/Augmented_Train', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_generator=val_datagen.flow_from_directory('data/validate', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 49535 images belonging to 19 classes.
Found 2179 images belonging to 19 classes.


In [0]:
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

In [0]:
reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, mode='auto')
early = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, mode='auto')

In [0]:
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model_final.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint("VGG16_SGD_MOMENTUM_feb_14.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [0]:
history = model_final.fit_generator(generator=train_generator,
                   steps_per_epoch=steps_per_epoch,
                   epochs=epochs,
                   workers=4,
                   validation_data=validation_generator, 
                   validation_steps=validation_steps,
                    callbacks=[reducer,checkpoint,early])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1335/1547 [========================>.....] - ETA: 8:39 - loss: 0.1854 - acc: 0.9481

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 1516 could not be retrieved. It could be because a worker has died.
  UserWarning)


1546/1547 [============================>.] - ETA: 2s - loss: 0.1655 - acc: 0.9538

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 21 could not be retrieved. It could be because a worker has died.
  UserWarning)


1547/1547 [==============================] - 4099s 3s/step - loss: 0.1654 - acc: 0.9539 - val_loss: 0.4828 - val_acc: 0.8653

Epoch 00001: val_loss improved from inf to 0.48278, saving model to VGG16_SGD_MOMENTUM_feb_14.h5
Epoch 2/100
   1/1547 [..............................] - ETA: 2:18 - loss: 0.0022 - acc: 1.0000

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.savefig('/content/gdrive/My Drive/FACEID_VIDEOS/Augmented/vgg_16.png')
plt.show()
